## This notebook covers the k-Exhange Algorithm for D-optimal experimental designs

Optimal experimental designs allow for fewer experiemntal runs compared to traditional factorial designs. One popular type of optimal design is D-optimal, which seeks to maximize the determinant of the information matrix $X^TX$, where $X$ is a subset of the full factorial design matrix of possible experimental runs. This is also equivalent to minimizing the determinant of the dispersion matrix $(X^TX)^{-1}$

In [1]:
import numpy as np

def calc_disp_matrix(design_matrix):
    return np.linalg.inv(design_matrix.T @ design_matrix)

This notebook will explore choosing a D-optimal design with 3 variables each containing 3 levels (-1, 0, 1) with full interactions between the variables plus an intercept. The model would take the form
$$x_1 + x_2 + x_3 + x_1^2 + x_ 1x_2 + x_1 x_3 + x_2^2 + x_2 x_3 + x_3^2 + \beta$$
 The full matrix is

In [13]:
import pandas as pd

df = pd.read_csv('3_fact_full_inter.csv').values
print(df)
print(f'Number of experiments in full factorial run = {df.shape[0]}')

[[-1. -1. -1.  1.  1.  1.  1.  1.  1.  1.]
 [-1. -1.  0.  1.  1. -0.  1. -0.  0.  1.]
 [-1. -1.  1.  1.  1. -1.  1. -1.  1.  1.]
 [-1.  0. -1.  1. -0.  1.  0. -0.  1.  1.]
 [-1.  0.  0.  1. -0. -0.  0.  0.  0.  1.]
 [-1.  0.  1.  1. -0. -1.  0.  0.  1.  1.]
 [-1.  1. -1.  1. -1.  1.  1. -1.  1.  1.]
 [-1.  1.  0.  1. -1. -0.  1.  0.  0.  1.]
 [-1.  1.  1.  1. -1. -1.  1.  1.  1.  1.]
 [ 0. -1. -1.  0. -0. -0.  1.  1.  1.  1.]
 [ 0. -1.  0.  0. -0.  0.  1. -0.  0.  1.]
 [ 0. -1.  1.  0. -0.  0.  1. -1.  1.  1.]
 [ 0.  0. -1.  0.  0. -0.  0. -0.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  1.  1.]
 [ 0.  1. -1.  0.  0. -0.  1. -1.  1.  1.]
 [ 0.  1.  0.  0.  0.  0.  1.  0.  0.  1.]
 [ 0.  1.  1.  0.  0.  0.  1.  1.  1.  1.]
 [ 1. -1. -1.  1. -1. -1.  1.  1.  1.  1.]
 [ 1. -1.  0.  1. -1.  0.  1. -0.  0.  1.]
 [ 1. -1.  1.  1. -1.  1.  1. -1.  1.  1.]
 [ 1.  0. -1.  1.  0. -1.  0. -0.  1.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  

A full factorial run of 3 variables with 3 levels would require 27 runs. Let's say we only had the money to complete 15 total runs. Choosing 15 runs out of 27 would give us a possible

In [3]:
from math import comb
print(f'Total number of possible combinations = {comb(27, 15)}!')

Total number of possible combinations = 17383860!


That is over 17 million possible combinations! Additionally, we must compute the determinant of each one of these possibilities. Numpy uses LU decomposition to compute the determinant with a run time of $O(n)^3$. Computationally it becomes impractical to enumerate all possible runs and compute the determinant. This is especially true when the number of experimental factors increases. Let's look at the number of possible experiemnts and time complexity if there were 5 varibles with 3 levels and we could only run 20 experiments

In [4]:
print(f'{3 ** 5} total runs for a full factorial experiment')
print(f'{comb(3 ** 5, 20)} total possible ways to pick 20 runs!')
print(f'{comb(3 ** 5, 20) * (20 ** 3)} total run time!')

243 total runs for a full factorial experiment
94833792242740173136768780680 total possible ways to pick 20 runs!
758670337941921385094150245440000 total run time!


Exchange algorithms start with randomly selecting a potential design matrix. The remaining design points are then sequentially swapped into the candidate design matrix and their effect on the determinant is calculated. If it increases, these experimental runs are swapped out.

We can first start out by calculating the variance of a single experimental run $\chi_x$. Note that we are defining $\chi_x$ as a $\textit{column}$ vector.
<br>
<br>
$$d(\chi_x) = \chi^{T}_x * (X^{T}_nX_n)^{-1} * \chi_x $$

In [5]:
def variance_single_point(design_point, dispersion_matrix):
    
    #Reshape for proper calc
    design_point = design_point.reshape((-1, 1))
    
    return (design_point.T @ dispersion_matrix @ design_point)[0][0]

In general, we want to see what impact adding and deleting points from the candidate design matrix has on the determinant. Adding experimental run $\chi_j$ increases the determinant proportionally by its variance $d(\chi_j)$
<br>
<br>
$$|X_{new}^T X_{new}| = |X_{old}^T X_{old}| * (1 + d(\chi_j))$$

This is a nice property because we do not have to compute the determinant each time. With the k-Exchange algorithm, we are swapping point $\chi_i$ in the candidate design matrix and potential run $\chi_j$. Similar to the equation above, we can see the impact of this swap on the determinant by 
<br>
<br>
$$ |X_{new}^T X_{new}| = |X_{old}^T X_{old}| * (1 + \Delta (\chi_i, \chi_j)) $$
<br>
<br>

where $\Delta (\chi_i, \chi_j))$ uses the combined varaince of $\chi_i$ and $\chi_j$
<br>
<br>


$$d(\chi_i, \chi_j) = \chi^{T}_i * (X^{T}_nX_n)^{-1} * \chi_j  = \chi^{T}_j * (X^{'}_nX_n)^{-1} * \chi_i $$



$$\Delta(\chi_i, \chi_j) = d(\chi_j) - [d(\chi_i)d(\chi_j) - (d(\chi_i, \chi_j))^2] - d(\chi_i)$$

In [25]:
def variance_combined(point_i, point_j, dispersion_matrix):

    point_i = point_i.reshape((-1, 1))
    point_j = point_j.reshape((-1, 1))
    
    return (point_i.T @ dispersion_matrix @ point_j)[0][0]


def delta(d_i, d_j, d_comb):

    return d_j - (d_i * d_j - d_comb ** 2) - d_i


def update_det(current_det, delta_ij):

    return current_det * (1 + delta_ij)

For the k-Exchange algorithm, we start by randomly selecting a design matrix. Next, we look to exchange the $k$ points with the lowest variance of prediction. $k$ is suggested to be selected as $k \leq \frac{n}{4}$ with $n$ being the number of desired experimental runs. The pseudo-code follows as 

Generate a random design <br>
While pairs of points $(\chi_i, \chi_j)$ with positive $\Delta(\chi_i, \chi_j)$ exist <br>
$\;\;\;\;$ Calculate $d(\chi_i)$ for all design points <br>
$\;\;\;\;$ Find the $k$ design points witht the lowest prediction variance <br>
$\;\;\;\;$ For design point $\chi_1$ to $\chi_k$ <br>
$\;\;\;\;\;\;\;\;$ Calculate the variance of predicition $d(\chi_i)$ <br>
$\;\;\;\;\;\;\;\;$ For support point $\chi_1$ to $\chi_N$ <br>
$\;\;\;\;\;\;\;\;\;\;\;\;$ Calculate the variance of prediction $d(\chi_j)$ for this support point<br>
$\;\;\;\;\;\;\;\;\;\;\;\;$ Calculate the combined variance of prediction $d(\chi_i, \chi_j)$ for this pair<br>
$\;\;\;\;\;\;\;\;\;\;\;\;$ Calculate the delta function $\Delta(\chi_i, \chi_j)$ for this pair<br>
$\;\;\;\;\;\;\;\;\;\;\;\;$ Select pair with the maximum $\Delta$ <br>
$\;\;\;\;\;\;\;\;\;\;\;\;$ If max delta > 0 then <br>
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ If two pairs have same max $\Delta$ <br>
$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$  Select pair randomly <br>
$\;\;\;\;\;\;\;\;\;\;\;\;$ Exchange point $\chi_i$ with $\chi_j$ <br>
$\;\;\;\;\;\;\;\;\;\;\;\;$ Update information and disperion matrix <br>
$\;\;\;\;\;\;\;\;\;\;\;\;$ Reset max delta <br>





Before coding this out, we'll define some helper functions to make this a little cleaner. <br><br> First we'll define a function for generating a random design matrix from a full factorial design matrix. We'll return a design matrix and its indicies from the full factorial matrix, and the left over "support points" and support indices

In [7]:
def random_design(full_matrix, n_runs):
    
    #Get random indicies for design matrix
    r, c = full_matrix.shape
    rand_ind = np.random.choice(np.arange(r), size = n_runs, replace = False)
    
    #Indicies for design and support points
    design_mask = ~np.ones(r, bool)
    design_mask[rand_ind] = True
    design_ind = np.arange(0, r)[design_mask]
    support_ind = np.arange(0, r)[~design_mask]
    
    #Get support points
    support_points = full_matrix[support_ind, :]
    
    #Get the design matrix
    design_mat = full_matrix[design_ind, :]
   
    return design_mat, design_ind, support_points, support_ind

Next we'll calculate the design variances for all points. We'll define a function for calculating the disperion matrix $(X^T X)^{-1}$

In [8]:
def calc_disp_matrix(design_matrix):
    
    return np.linalg.inv(design_matrix.T @ design_matrix)

Next, instead of looping through and calculating $d(\chi_x) = \chi^{T}_x * (X^{T}_nX_n)^{-1} * \chi_x $ for each $\chi_x$, we can do this in one shot by computing <br>
 $$diag(X^T * (X^{T}_nX_n)^{-1} * X)$$

In [9]:
def design_variances(design_matrix, dispersion_mat):

    return np.diag(design_matrix @ dispersion_mat @ design_matrix.T)

Now we'll define a function for finding the k lowest variances of the design matrix

In [10]:
def k_lowest_var_ind(variances, k):

    return np.sort(np.argsort(variances)[:k])

Time to put it all together! One important thing to note is that this alogrithm and other coordinate exchange algorithms are not guaranteed to find the *exact* optimal design. The output of these alogrithms greatly depends on the randomly chosen design matrix at the beginning of the alorgithm. For this reason, we'll specifiy the number of times to run the k-Exchange alorgithm in our function to potentially find better designs. We'll also define the maximum number of iterations to look for potential points to swap out in the algorthim to speed things up if desired.

In [67]:
def k_exchange(full_factorial_matrix, desired_num_experiments, num_runs, seed, maximum_iterations):

    #Define k 
    k = round(desired_num_experiments / 4)

    #Keep track of all determinants and indicies to find the best run
    d = {}
    
    #Number of times to run the algorithm
    for run in range(num_runs):

        #Chnage seed each run to get different output for each run
        np.random.seed(seed + run)

        #Generate random design
        X, X_ind, support_points, support_ind = random_design(full_factorial_matrix, desired_num_experiments)

        #It is possible the randomly generated design has orthogonal columns making the determinant zero. In this case, we'll skip this iteration
        if np.linalg.det(X.T @ X) == 0:
            continue

        else:

            #Calculate dispersion matrix
            X_disp = calc_disp_matrix(X)

            #Whilte couples with positive delta are found
            max_delta = 1
            max_iters = 0

            while max_delta > 0 and max_iters < maximum_iterations:

                maximum_deltas = []
                #Calculate the vriance of prediction d(x_i) for all design points
                dv = design_variances(X, X_disp)

                #Select k design points with the lowest variance
                k_ind = k_lowest_var_ind(dv, k)
                k_design_points = X[k_ind]

                #For design point x_1 to x_k do
                for i in range(k):

                    #Calculate the varaince of prediction d(x_i) for this point
                    x_i = k_design_points[0, :]
                    dx_i = variance_single_point(x_i, X_disp)

                    #For support point x_1 to x_N do
                    couple_deltas = []
                    for j in range(len(support_ind)):

                        #Calculate the variance of prediction for this support point
                        x_j = support_points[j, :]
                        dx_j = variance_single_point(x_j, X_disp)

                        #Calculate the variance of prediction for this couple
                        dx_ij_comb = variance_combined(x_i, x_j, X_disp)

                        #Calculate the delta function for this couple
                        x_ij_delta = delta(dx_i, dx_j, dx_ij_comb)
                        couple_deltas.append(x_ij_delta)

                    #Select couple with max delta
                    delta_i = couple_deltas[np.argmax(couple_deltas)]

                    #If max delta positive exchange point x_i and x_j
                    if delta_i > 0:
                        ind_j = np.argmax(couple_deltas)
                        support_exchange = support_ind[ind_j]
                        design_exchange = X_ind[k_ind[i]]
                        support_ind[ind_j] = design_exchange
                        X_ind[np.where(X_ind == design_exchange)[0][0]] = support_exchange

                        #Update information matrix
                        X = full_factorial_matrix[X_ind]

                        #Case for if calculating the dispersion matrix produces a singular matrix
                        try:
                            X_disp = calc_disp_matrix(X)
                        except:
                            continue
                        
                    maximum_deltas.append(delta_i)
                max_delta = np.max(maximum_deltas)
                max_iters += 1

        d[int(np.linalg.det(X.T @ X))] = X_ind

    return full_factorial_matrix[[v for k, v in sorted(d.items(), key = lambda x: x[0], reverse = True)][0]]

Let's see our optimal design!

In [79]:
X = k_exchange(df, 15, 1000, 44, 1000)
print('Optimal Design is')
print(X)


Optimal Design is
[[-1. -1.  1.  1.  1. -1.  1. -1.  1.  1.]
 [-1.  0. -1.  1. -0.  1.  0. -0.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [-1. -1. -1.  1.  1.  1.  1.  1.  1.  1.]
 [-1.  1. -1.  1. -1.  1.  1. -1.  1.  1.]
 [ 1. -1.  1.  1. -1.  1.  1. -1.  1.  1.]
 [-1.  1.  1.  1. -1. -1.  1.  1.  1.  1.]
 [ 0. -1. -1.  0. -0. -0.  1.  1.  1.  1.]
 [-1. -1.  0.  1.  1. -0.  1. -0.  0.  1.]
 [ 1. -1. -1.  1. -1. -1.  1.  1.  1.  1.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  1.  1.]
 [ 0.  1. -1.  0.  0. -0.  1. -1.  1.  1.]
 [ 0.  1.  0.  0.  0.  0.  1.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.  0.  1.]
 [ 1.  1. -1.  1.  1. -1.  1. -1.  1.  1.]]
